# Performance Benchmark

🚧 Under construction 🚧

In [79]:
import pandas as pd
import numpy as np
from timeit import timeit
import tabulate

import stringcompare
import jellyfish
import py_stringmatching

names = pd.read_csv("data/REQ-names.csv", header=None)[0].values

## Levenshtein distance

In [90]:
print("stringcompare: \t\t", timeit(lambda: stringcompare.Levenshtein().pairwise(names, names), number=1))
print("jellyfish: \t\t", timeit(lambda: np.array([jellyfish.levenshtein_distance(s, t) for s in names for t in names]), number=1))
cmp = py_stringmatching.Levenshtein()
print("py_stringmatching: \t", timeit(lambda: [cmp.get_raw_score(s, t) for s in names for t in names], number=1))

stringcompare: 		 1.3243956130008883
jellyfish: 		 2.030931817000237
py_stringmatching: 	 33.7859749069994


## Damerau-Levenshtein distance

In [93]:
print("stringcompare: \t\t", timeit(lambda: stringcompare.DamerauLevenshtein().pairwise(names, names), number=1))
print("jellyfish: \t\t", timeit(lambda: [jellyfish.damerau_levenshtein_distance(s, t) for s in names for t in names], number=1))

stringcompare: 		 2.7019762420004554
jellyfish: 		 4.423821354999745


## Jaro distance

In [95]:
print("stringcompare: \t\t", timeit(lambda: stringcompare.Jaro().pairwise(names, names), number=1))
print("jellyfish: \t\t", timeit(lambda: [jellyfish.jaro_similarity(s, t) for s in names for t in names], number=1))

stringcompare: 		 1.343546205000166
jellyfish: 		 1.3307045370002015


## Jaro-Winkler distance

In [94]:
print("stringcompare: \t\t", timeit(lambda: stringcompare.JaroWinkler().pairwise(names, names), number=1))
print("jellyfish: \t\t", timeit(lambda: [jellyfish.jaro_winkler_similarity(s, t) for s in names for t in names], number=1))

stringcompare: 		 1.35290373999851
jellyfish: 		 1.4211493950006115
